In [1]:
# All imports here
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Activation, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [2]:
# Read the train features from CSV
train_features = pd.read_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/train-features.csv")

In [3]:
# Read the train labels from CSV
train_labels = pd.read_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/train-output.csv")

In [4]:
# Initialize Global variables
scaler = StandardScaler()

In [5]:
train_features.shape

(348607, 55)

In [6]:
train_features.describe()

,Id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
count,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.00000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000,348607.000000
mean,174303.000000,2959.536458,155.768332,14.110161,269.624006,46.564036,2350.054468,212.140224,223.325863,142.538348,1980.192825,0.448657,0.051336,0.436623,0.063384,0.005295,0.013035,0.008270,0.021442,0.002803,0.011302,0.000181,0.000307,0.001971,0.056132,0.021468,0.051686,0.030057,0.000964,0.000009,0.004819,0.005832,0.003322,0.006902,0.015717,0.001345,0.057165,0.09982,0.036580,0.000806,0.004426,0.001922,0.001667,0.198335,0.051712,0.044586,0.090242,0.077615,0.002834,0.003290,0.000215,0.000565,0.026769,0.023674,0.014919
std,100634.316983,279.980511,111.910643,7.488525,212.688171,58.459698,1558.087024,26.763785,19.782477,38.307210,1323.117587,0.497358,0.220682,0.495968,0.243652,0.072576,0.113423,0.090563,0.144854,0.052865,0.105709,0.013442,0.017517,0.044349,0.230177,0.144940,0.221392,0.170744,0.031031,0.002934,0.069253,0.076143,0.057539,0.082790,0.124378,0.036654,0.232157,0.29976,0.187728,0.028380,0.066382,0.043798,0.040790,0.398746,0.221444,0.206393,0.286529,0.267565,0.053161,0.057266,0.014666,0.023765,0.161409,0.152032,0.121230
min,0.000000,1860.000000,0.000000,0.000000,0.000000,-166.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,87151.500000,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,174303.000000,2996.000000,127.000000,13.000000,218.000000,30.000000,1998.000000,218.000000,226.000000,143.000000,1711.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,261454.500000,3164.000000,261.000000,18.0

In [7]:
# Drop feature_28 as it is not useful
def drop_feature_28(df):
  return df.drop(columns=["feature_28"])

In [8]:
# Drop feature 28
train_features = drop_feature_28(train_features)

In [9]:
train_features.columns

Index(['Id', 'feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13', 'feature_14',
       'feature_15', 'feature_16', 'feature_17', 'feature_18', 'feature_19',
       'feature_20', 'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_29', 'feature_30',
       'feature_31', 'feature_32', 'feature_33', 'feature_34', 'feature_35',
       'feature_36', 'feature_37', 'feature_38', 'feature_39', 'feature_40',
       'feature_41', 'feature_42', 'feature_43', 'feature_44', 'feature_45',
       'feature_46', 'feature_47', 'feature_48', 'feature_49', 'feature_50',
       'feature_51', 'feature_52', 'feature_53'],
      dtype='object')

In [10]:
# Normalize the numeric features
def normalize_features(df, only_transform=False):

  original_order = df.columns

  # Get the columns to be normalized
  columns = [f"feature_{idx}" for idx in range(0,10)]

  # Normalize Data
  if not only_transform:
    transformed_data = scaler.fit_transform(df[columns])
  else:
    transformed_data = scaler.transform(df[columns])
  
  # Convert to DF
  normalized_df = pd.DataFrame(transformed_data, columns=columns)

  # Concat the df's again
  df = pd.concat([normalized_df, df.loc[:, ~df.columns.isin(columns)]], axis=1)
  df = df[original_order]

  return df

In [11]:
# Normalize features 0 to 9
normalized_train_df = normalize_features(train_features)

In [12]:
normalized_train_df.head(5)

,Id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
0,0,-0.005488,-0.632366,-0.815938,-1.126646,-0.796516,1.482554,0.742040,0.135177,-0.431730,0.289323,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0.080233,0.806284,-0.682400,0.725834,0.452209,2.415110,-0.416243,1.196724,1.082347,2.483386,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2,-0.566242,0.270142,0.652978,0.434327,0.999595,-0.965964,0.293672,1.348374,0.325309,-0.040958,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,0.758852,0.189720,0.118827,-0.364967,-0.557035,-0.008379,0.517856,1.196724,0.194785,-0.851167,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,-0.959127,-1.025537,-0.548862,-0.195705,-0.351765,-0.405661,0.368400,-0.319771,-0.327311,3.209702,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
train_labels.head(5)

,Id,Covtype
0,0,1
1,1,2
2,2,5
3,3,1
4,4,2


In [14]:
oversampled_train_df =  normalized_train_df.drop(columns=['Id']).append(normalized_train_df.drop(columns=['Id'])).append(normalized_train_df.drop(columns=['Id']))
oversampled_labels = np.concatenate((train_labels['Covtype'],train_labels['Covtype'], train_labels['Covtype']), axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(normalized_train_df.drop(columns=['Id']), train_labels['Covtype'], test_size=0.10)

In [ ]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

313746 34861 313746 34861


In [ ]:
np.unique(y_train)

array([1, 2, 3, 4, 5, 6, 7])

In [ ]:
X_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
61281,0.683847,1.262006,0.385903,-0.952682,-0.676776,1.051898,-1.574526,0.438476,1.552234,-0.259382,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
190324,1.287461,-1.391901,-0.415324,0.232152,0.691691,0.413935,-0.341515,-0.218672,0.273099,0.268160,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
163980,-1.991343,-0.873630,0.519441,-1.126646,-0.625458,-1.214346,0.667312,-1.280219,-1.267083,-1.159530,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
24681,-0.391229,-0.230258,1.320668,0.815166,0.435103,-0.209266,1.451956,-0.319771,-1.658655,-0.028110,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172984,0.451688,-0.927244,0.252365,0.232152,-0.471506,0.853577,0.555220,-1.027470,-0.979930,0.500188,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155820,-0.548383,0.216527,-1.350089,0.429625,-0.557035,-0.104009,0.331036,0.943975,0.351414,0.033109,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
56453,0.826714,-0.453651,-1.216551,1.637968,0.828537,-0.539158,0.629948,0.489026,-0.118473,0.379262,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
70992,0.258817,1.690920,-0.949476,-0.985595,-0.728093,-0.872901,-0.192059,0.337377,0.481937,0.078457,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
291088,-0.391229,1.306684,-1.350089,-1.267698,-0.796516,0.343335,-0.192059,0.741776,0.690776,0.509258,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
y_train

61281     1
190324    1
163980    6
24681     2
172984    1
         ..
155820    2
56453     1
70992     2
291088    2
184471    3
Name: Covtype, Length: 313746, dtype: int64

## NN Model

In [15]:
num_input_features = (53, )
num_classes = 7

In [16]:
temp_y_train = oversampled_labels - 1

In [17]:
y_train_encoded = to_categorical(temp_y_train, num_classes)

In [ ]:
def build_model():
  model = Sequential()
  model.add(Dense(900, input_shape=num_input_features, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(750, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(500, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(250, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(100, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(50, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(7, activation='softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
model = build_model()

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 900)               48600     
                                                                 
 dropout (Dropout)           (None, 900)               0         
                                                                 
 dense_1 (Dense)             (None, 750)               675750    
                                                                 
 dropout_1 (Dropout)         (None, 750)               0         
                                                                 
 dense_2 (Dense)             (None, 500)               375500    
                                                                 
 dropout_2 (Dropout)         (None, 500)               0         
                                                                 
 dense_3 (Dense)             (None, 250)               1

In [ ]:
# checkpoint_filepath = '/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model_111321_best'
# model_checkpoint_callback = ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     monitor='val_accuracy',
#     mode='max',
#     verbose=1,
#     save_best_only=True)

In [ ]:
model.fit(oversampled_train_df , y_train_encoded, batch_size=128, epochs=10)

Epoch 1/40
5447/5447 [==============================] - 196s 36ms/step - loss: 0.1379 - accuracy: 0.9452
Epoch 2/40
5447/5447 [==============================] - 196s 36ms/step - loss: 0.1353 - accuracy: 0.9460
Epoch 3/40
5447/5447 [==============================] - 196s 36ms/step - loss: 0.1332 - accuracy: 0.9470
Epoch 4/40
5447/5447 [==============================] - 196s 36ms/step - loss: 0.1313 - accuracy: 0.9477
Epoch 5/40
5447/5447 [==============================] - 198s 36ms/step - loss: 0.1291 - accuracy: 0.9487
Epoch 6/40
5447/5447 [==============================] - 209s 38ms/step - loss: 0.1274 - accuracy: 0.9497
Epoch 7/40
5447/5447 [==============================] - 200s 37ms/step - loss: 0.1246 - accuracy: 0.9511
Epoch 8/40
5447/5447 [==============================] - 200s 37ms/step - loss: 0.1238 - accuracy: 0.9511
Epoch 9/40
5447/5447 [==============================] - 201s 37ms/step - loss: 0.1222 - accuracy: 0.9521
Epoch 10/40
5447/5447 [==============================] 

KeyboardInterrupt: ignored

In [ ]:
model.save("/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model_111321_9591.hd5")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model_111321_9591.hd5/assets


In [ ]:
y_predicted = model.predict(X_test)

In [ ]:
y_pred = np.argmax(y_predicted, axis=1) 

In [ ]:
np.unique(y_pred)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
y_pred = y_pred + 1

In [ ]:
f1_score(y_test, y_pred, average='macro')

0.8874169915381909

## Load test data

In [21]:
test_path = "/content/drive/MyDrive/CSCI 5622 - ML/PS4/test-features.csv"

In [22]:
test_df = pd.read_csv(test_path)

In [23]:
test_df = drop_feature_28(test_df)

In [24]:
normalize_test_df = normalize_features(test_df, only_transform=True)

In [25]:
normalize_test_df

,Id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53
0,0,0.476689,-0.909373,-0.415324,-0.952682,-0.642564,1.610917,0.517856,-0.370321,-0.536149,0.334670,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,1,1.037443,0.189720,-1.350089,0.725834,-0.403082,1.331728,0.368400,0.943975,0.351414,0.138164,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,2,0.608842,0.028877,-0.682400,-0.637667,-1.001786,1.398477,0.667312,0.943975,0.012051,-0.695474,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,-1.966341,0.413113,-0.415324,-1.126646,-0.762305,-0.741971,0.106853,1.348374,0.664671,-0.752914,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0.312392,-0.185580,0.385903,-1.126646,-0.745199,0.630226,1.190408,0.337377,-0.875511,0.715589,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232400,232400,0.758852,-0.748530,-1.216551,0.001768,0.349574,0.180956,0.480492,0.387927,-0.040158,-0.158106,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
232401,232401,0.130236,1.547949,-0.415324,1.529829,0.674585,-0.532098,-0.715155,0.185727,0.795195,0.197116,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
232402,232402,-0.230504,-1.257865,0.118827,-1.126646,-0.676776,-0.320300,-0.229423,-0.724170,-0.118473,-0.489899,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
232403,232403,1.448187,-0.560880,-0.682400,0.034680,-0.060966,-0.241357,0.854132,0.135177,-0.562254,1.008836,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [26]:
true_y_pred = classifer.predict(normalize_test_df.drop(columns=["Id"]))

NameError: ignored

In [ ]:
len(true_y_pred)

232405

In [ ]:
true_y_pred

array([2, 7, 1, ..., 2, 7, 2])

In [ ]:
result_df = pd.DataFrame(list(zip(test_df.loc[:, "Id"], true_y_pred )), columns=["Id", "Covtype"])

In [ ]:
result_df

,Id,Covtype
0,0,2
1,1,7
2,2,1
3,3,3
4,4,1
...,...,...
232400,232400,1
232401,232401,2
232402,232402,2
232403,232403,7


In [ ]:
result_df.to_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/rf_submission.csv", index=False)

In [ ]:
result_df['Covtype'].unique()

array([2, 7, 1, 3, 5, 6, 4])

## NN experiment

In [ ]:
def build_model3(num_input_features):
  model = Sequential()
  model.add(Dense(450, input_shape=num_input_features, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(250, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(100, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(75, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(65, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(50, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(25, activation='swish'))
  model.add(Dropout(0.1))
  model.add(Dense(15, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(7, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
oversampled_train_df = normalized_train_df.drop(columns='Id').append(normalized_train_df.drop(columns='Id'))

In [ ]:
oversampled_train_df.shape

(697214, 53)

In [ ]:
oversampled_labels = np.concatenate((train_labels['Covtype'], train_labels['Covtype']), axis=0) - 1

In [ ]:
oversampled_labels.shape

(697214,)

In [ ]:
oversampled_labels = to_categorical(oversampled_labels, 7)

In [ ]:
model3 = build_model3((53,))
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 450)               24300     
                                                                 
 dropout_16 (Dropout)        (None, 450)               0         
                                                                 
 dense_19 (Dense)            (None, 250)               112750    
                                                                 
 dropout_17 (Dropout)        (None, 250)               0         
                                                                 
 dense_20 (Dense)            (None, 100)               25100     
                                                                 
 dropout_18 (Dropout)        (None, 100)               0         
                                                                 
 dense_21 (Dense)            (None, 75)               

In [ ]:
model3.fit(oversampled_train_df, oversampled_labels, epochs=100)

Epoch 1/100
21788/21788 [==============================] - 107s 5ms/step - loss: 0.5084 - accuracy: 0.7899
Epoch 2/100
21788/21788 [==============================] - 104s 5ms/step - loss: 0.3701 - accuracy: 0.8508
Epoch 3/100
21788/21788 [==============================] - 105s 5ms/step - loss: 0.3193 - accuracy: 0.8731
Epoch 4/100
21788/21788 [==============================] - 105s 5ms/step - loss: 0.2925 - accuracy: 0.8853
Epoch 5/100
21788/21788 [==============================] - 104s 5ms/step - loss: 0.2768 - accuracy: 0.8920
Epoch 6/100
21788/21788 [==============================] - 104s 5ms/step - loss: 0.2647 - accuracy: 0.8975
Epoch 7/100
21788/21788 [==============================] - 105s 5ms/step - loss: 0.2554 - accuracy: 0.9013
Epoch 8/100
21788/21788 [==============================] - 105s 5ms/step - loss: 0.2492 - accuracy: 0.9037
Epoch 9/100
21788/21788 [==============================] - 107s 5ms/step - loss: 0.2442 - accuracy: 0.9058
Epoch 10/100
21788/21788 [===========

In [ ]:
model3.save("/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model3.hd5")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model3.hd5/assets


## Load Saved Model and test


In [18]:
model = load_model("/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model_9609.hd5")

In [34]:
y_pred = model.predict(normalize_test_df.drop(columns=["Id"]))

In [35]:
y_pred = np.argmax(y_pred, axis=1) + 1

In [36]:
np.unique(y_pred)

array([1, 2, 3, 4, 5, 6, 7])

In [37]:
result_df = pd.DataFrame(list(zip(test_df.loc[:, "Id"], y_pred)), columns=["Id", "Covtype"])

In [38]:
result_df.to_csv("/content/drive/MyDrive/CSCI 5622 - ML/PS4/NN_9614Model_submission.csv", index=False)

In [33]:
# Retrain
model.fit(oversampled_train_df , y_train_encoded, batch_size=128, epochs=2)

Epoch 1/2
8171/8171 [==============================] - 272s 33ms/step - loss: 0.1004 - accuracy: 0.9622
Epoch 2/2
8171/8171 [==============================] - 272s 33ms/step - loss: 0.1029 - accuracy: 0.9614


In [32]:
model.save("/content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model_9620.hd5")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/CSCI 5622 - ML/PS4/models/NN_model_9620.hd5/assets
